# 1. linux 内核模块
linux 内核模块三要素：入口、出口、许可证

## 1.1 模块编写基本

In [ ]:
#include <linux/init.h>
#include <linux/module.h>
// 入口
// static :限定作用域
// int    :函数的返回值类型
// __init :内核中定义的宏，#define __init __section(".init.text") 
// .init.text是一个段，在内核链接脚本（vmlinux.lds）中可以找到它
// __init告诉编译器将demo_init函数放在.init.text段中
// demo_init:入口函数的名字，习惯上入口命名格式：led_init adc_init uart_init...
// (void) :没有参数
static int __init demo_init(void)
{
    return 0;
}
static void __exit demo_exit(void)
{
}
module_init(demo_init); //告诉内核入口地址
module_exit(demo_exit); //告诉内核出口地址
MODULE_LICENSE("GPL");  // 许可证

## 1.2 接口

In [ ]:
/**
 * @功能：打印函数
 * @参数：fmt 格式化字符串
 * @返回值：打印字符的个数
 * @备注：可变参数
*/
int printk(const char *fmt, ...);

In [ ]:
/**
 * @功能：接受命令的参数
 * @参数：name  参数名， type 参数类型， perm 参数权限
 * @返回值：无
 * @备注：无
*/
moudle_param(name, type, perm)

/**
 * @功能：对变量进行描述
 * @参数：_parm 变量名， desc：描述
 * @返回值：无
 * @备注：无
*/
MODULE_PARM_DESC(_parm, desc)

/**
 * @功能：导出符号表
 * @参数：add 函数名
 * @返回值：无
 * @备注：无
*/
EXPORT_SYMBOL_GPL(add)

# 2. 字符设备驱动
按照字节流访问，只能顺序访问，不能无序访问的设备。

## 2.1 接口

In [ ]:
#include <linux/fs.h>
/**
 * @功能：注册字符设备驱动
 * @参数：major：主设备号，name：设备名，fops：文件操作集
 * @返回值：成功返回0，失败返回负值
 * @备注：如果major为0，则由系统自动分配主设备号
*/
int register_chrdev(unsigned int major, const char *name, const struct file_operations *fops)

/**
 * @功能：注销字符设备驱动
 * @参数：major：主设备号，name：设备名
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev(unsigned int major, const char *name)

In [ ]:
#include <linux/uaccess.h>
/**
 * @功能：从用户空间拷贝数据到内核空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

/**
 * @功能：从内核空间拷贝数据到用户空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

In [ ]:
#include <linux/io.h>
/**
 * @功能：将物理地址映射到内核虚拟地址空间
 * @参数：phy_addr：物理地址，size：映射字节数
 * @返回值：映射成功返回内核虚拟地址，失败返回NULL
 * @备注：无
*/
void  *ioremap(resource_size_t phy_addr, size_t size)

/**
 * @功能：解除内核虚拟地址空间的映射
 * @参数：virt_addr：内核虚拟地址
 * @返回值：无
 * @备注：无
*/
void iounmap(volatile void  *virt_addr)

In [ ]:
#include <linux/device.h>
/**
 * @功能：创建一个class
 * @参数：owner：模块的拥有者，name：class的名字
 * @返回值：成功返回class指针，失败返回NULL
 * @备注：无
*/
struct class * class_create(owner, name)
/**
 * @功能：注销一个class
 * @参数：cls：class指针
 * @返回值：无
 * @备注：无
*/
void class_destroy(struct class *cls)

/**
 * @功能：创建一个device
 * @参数：class：class指针，parent：父设备指针，devt：设备号，drvdata：驱动数据，fmt：设备名，...：变参
 * @返回值：成功返回device指针，失败返回NULL
 * @备注：fmt,... - 设备名，如 "myled%d"，%d - myled0, myled1, ...
*/
struct device *device_create(struct class *class, struct device *parent, dev_t devt, void *drvdata, const char *fmt, ...)
/**
 * @功能：注销一个device
 * @参数：class：class指针，devt：设备号
 * @返回值：无
 * @备注：无
*/
void device_destroy(struct class *class, dev_t devt)

In [ ]:
#include <linux/cdev.h>
/**
 * @功能：分配一个cdev对象
 * @参数：无
 * @返回值：成功返回cdev指针，失败返回NULL
 * @备注：无
*/
struct cdev *cdev_alloc(void)

/**
 * @功能：初始化一个cdev对象
 * @参数：cdev：cdev指针，fops：文件操作集
 * @返回值：无
 * @备注：无
*/
void cdev_init(struct cdev *cdev, const struct file_operations *fops)

/**
 * @功能：添加一个cdev对象到系统
 * @参数：p：cdev指针，dev：设备号，count：设备号个数
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int cdev_add(struct cdev *p, dev_t dev, unsigned count)

/**
 * @功能：注册指定的设备号
 * @参数：from: 起始设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：主设备号不能超过511，
*/
int register_chrdev_region(dev_t from, unsigned count, const char *name)
/**
 * @功能：注销指定的设备号
 * @参数：from: 起始设备号，count：设备号个数
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev_region(dev_t from, unsigned count)

/**
 * @功能: 动态注册设备号
 * @参数：dev：申请到的设备号指针，firstminor：起始次设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int alloc_chrdev_region(dev_t *dev, unsigned int firstminor, unsigned int count, const char *name)


## 2.2 结构体

In [ ]:
// 文件操作结构体
// struct file_operations {
//     struct module *owner;
//     loff_t (*llseek) (struct file *, loff_t, int);
//     ssize_t (*read) (struct file *, char __user *, size_t, loff_t *);
//     ssize_t (*write) (struct file *, const char __user *, size_t, loff_t *);
//     ssize_t (*aio_read) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
//     ssize_t (*aio_write) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
//     int (*readdir) (struct file *, void *, filldir_t);
//     unsigned int (*poll) (struct file *, struct poll_table_struct *);
//     long (*unlocked_ioctl) (struct file *, unsigned int, unsigned long);
//     long (*compat_ioctl) (struct file *, unsigned int, unsigned long);
//     int (*mmap) (struct file *, struct vm_area_struct *);
//     int (*open) (struct inode *, struct file *);
//     int (*flush) (struct file *, fl_owner_t id);
//     int (*release) (struct inode *, struct file *);
//     int (*fsync) (struct file *, loff_t, loff_t, int datasync);
//     int (*aio_fsync) (struct kiocb *, int datasync);
//     int (*fasync) (int, struct file *, int);
//     int (*lock) (struct file *, int, struct file_lock *);
//     ssize_t (*sendpage) (struct file *, struct page *, int, size_t, loff_t *, int);
//     unsigned long (*get_unmapped_area)(struct file *, unsigned long, unsigned long, unsigned long, unsigned long);
//     int (*check_flags)(int);
//     int (*flock) (struct file *, int, struct file_lock *);
//     ssize_t (*splice_write)(struct pipe_inode_info *, struct file *, loff_t *, size_t, unsigned int);
//     ssize_t (*splice_read)(struct file *, loff_t *, struct pipe_inode_info *, size_t, unsigned int);
//     int (*setlease)(struct file *, long, struct file_lock **, void **);
//     long (*fallocate)(struct file *file, int mode, loff_t offset, loff_t len);
// };

In [ ]:
// 文件结构体
// struct file {
//     struct path     f_path;
//     struct inode    *f_inode;       /* cached value */
//     const struct file_operations    *f_op;
//     spinlock_t      f_lock;         /* f_ep_links, f_flags, no IRQ */
//     atomic_long_t   f_count;
//     unsigned int    f_flags;
//     fmode_t         f_mode;
//     loff_t          f_pos;
//     struct fown_struct f_owner;
//     const struct cred *f_cred;
//     struct file_ra_state    f_ra;
//     u64             f_version;
// #ifdef CONFIG_SECURITY
//     void            *f_security;
// #endif
//     /* needed for tty driver, and maybe others */
//     void            *private_data;
// #ifdef CONFIG_EPOLL
//     /* Used by fs/eventpoll.c to link all the hooks to this file */
//     struct list_head    f_ep_links;
//     struct list_head    f_tfile_llink;
// #endif /* #ifdef CONFIG_EPOLL */
//     struct address_space    *f_mapping;
// #ifdef CONFIG_DEBUG_WRITECOUNT
//     unsigned long f_mnt_write_state;
// #endif
// };

# 3. 内核中的并发和竞态
当多个进程通过同一个驱动（临界资源）访问硬件的时候，竞态就会产生。有 5 种解决方案来优化程序。

## 3.1 中断屏蔽
暂时的关闭中断，只针对单核处理器有效。

In [ ]:
local_irq_disable();
// 临界区
local_irq_enable();

## 3.2 自旋锁（忙等锁）
除首次获得锁之外的进程尝试获取锁会进入自旋状态直到获得锁的进程释放锁。

In [ ]:
// 由于自旋锁内的代码不能太长，所以常用来加锁一个信号量
int flag;
// 1. 定义一个自旋锁
spinlock_t my_lock;
// 2. 初始化自旋锁
/**
 * @功能：初始化自旋锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_lock_init(spinlock_t *lock);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_lock(spinlock_t *lock);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_unlock(spinlock_t *lock);

## 3.3 信号量
除首次获得信号量之外的进程尝试获取信号量会进入休眠状态直到获得获得信号量的进程释放信号量，休眠不消耗消耗 CPU 资源。

In [ ]:
// 1. 定义一个信号量
struct semaphore my_sem;
// 2. 初始化信号量
/**
 * @功能：初始化信号量
 * @参数：sem：信号量指针，val：信号量初始值
 * @返回值：无
 * @备注：val一般为1，表示只有一个进程可以访问临界区
*/
void sema_init(struct semaphore *sem, int val);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：sem：信号量指针
 * @返回值：无
 * @备注：让信号量的值减1，如果值为0，则休眠等待
*/
void down(struct semaphore *sem);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：sem：信号量指针
 * @返回值：无
 * @备注：让信号量的值加1，如果值为0，则唤醒等待的进程
*/
void up(struct semaphore *sem);
// 5. 尝试加锁
/**
 * @功能：尝试加锁
 * @参数：sem：信号量指针
 * @返回值：成功返回0，失败返回1
 * @备注：不会休眠，相当于非阻塞
*/
int down_trylock(struct semaphore *sem);

## 3.4 互斥体
互斥体与信号量的区别是在获取不到资源的情况下，会稍微等一会再进入休眠。  
通常在不确定临界区大小的时候，优先使用互斥体。

In [ ]:
// 1. 定义一个互斥锁
struct mutex my_mutex;
// 2. 初始化互斥锁
/**
 * @功能：初始化互斥锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_init(struct mutex *lock);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_lock(struct mutex *lock);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_unlock(struct mutex *lock);
// 5. 尝试加锁
/**
 * @功能：尝试加锁
 * @参数：lock：互斥锁指针
 * @返回值：成功返回1，失败返回0
 * @备注：无
*/
int mutex_trylock(struct mutex *lock);

## 3.5 原子操作
相当于将整个操作看成一个不可被分割的状态，只要开始执行就必须一次执行完。对应原子变量同时只能有一个核处理。在一个核上运行的时候修改 counter 是通过内联汇编完成的。

In [ ]:
typedef struct {
    int counter;
} atomic_t;

In [ ]:
// 1. 定义一个原子变量
atomic_t my_atomic;
// 2. 初始化原子变量
ATOMIC_INIT(my_atomic);
// 3. 读取原子变量的值
/**
 * @功能：读取原子变量的值
 * @参数：v：原子变量指针
 * @返回值：原子变量的值
 * @备注：无
*/
int atomic_read(atomic_t *v);
// 4. 设置原子变量的值
/**
 * @功能：设置原子变量的值
 * @参数：v：原子变量指针，i：要设置的值
 * @返回值：无
 * @备注：无
*/
void atomic_set(atomic_t *v, int i);
// 5. 原子变量加1
/**
 * @功能：原子变量加1
 * @参数：v：原子变量指针
 * @返回值：无
 * @备注：无
*/
void atomic_inc(atomic_t *v);
// 6. 原子变量减1
/**
 * @功能：原子变量减1
 * @参数：v：原子变量指针
 * @返回值：无
 * @备注：无
*/
void atomic_dec(atomic_t *v);
// 

# 4. IO模型

## 4.1 非阻塞IO
调用函数读取数据的时候，不管数据是否准备好函数都会立即返回。

## 4.2 阻塞IO模型
调用函数读取数据的时候，如果数据准备好了就会立即返回，如果数据没有准备好则进程休眠。

In [ ]:
// 1. 定义信号
unsigned int flag;
// 2. 定义等待队列头
wait_queue_head_t my_queue;
// 3. 初始化等待队列头
init_waitqueue_head(&my_queue);
// 4. 进程加入等待队列
wait_event(my_queue, flag);
// 5. 进程加入等待队列，超时
wait_event_timeout(my_queue, flag, 1000);
// 6. 进程加入等待队列，中断
wait_event_interruptible(my_queue, flag);
// 7. 进程加入等待队列，中断，超时
wait_event_interruptible_timeout(my_queue, flag, 1000);
// 8. flag置为1
flag = 1;
// 9. 唤醒等待队列头
wake_up(&my_queue);
// 10. 唤醒等待队列头，中断
wake_up_interruptible(&my_queue);


## 4.3 IO多路复用
在同一个app应用程序中如果想要同时监听多个fd对应数据。就需要使用select/poll/epoll来完成监听。如果所有的文件描述符的数据都没有准备好，此时进程休眠。如果有一个或者多个硬件的数据准备好就会，产生硬件中断，在处理函数中唤醒休眠的进程。此时select/poll/epoll就会返回，从就绪的表中找到准备好数据的文件描述符，然后调用read将数据读取到用户空间即可。

In [ ]:
// 1. 定义返回值
__poll_t mask = 0;
// 2. 阻塞等待
poll_wait(file, &my_queue, wait);
// 3. 判断是否可读
if (flag) {
    mask |= POLLIN | POLLRDNORM; //EPOLLIN 读  EPOLLOUT写
}
// 4. 返回
return mask;

# 5. 设备树
